# 掩码语言模型训练实例

## Step1 导入相关包

In [1]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForMaskedLM, DataCollatorForLanguageModeling, TrainingArguments, Trainer

## Step2 加载数据集

In [2]:
ds = Dataset.load_from_disk('./data/wiki_cn_filtered')
ds

Dataset({
    features: ['source', 'completion'],
    num_rows: 10000
})

In [5]:
type(ds['completion'])

list

## Step3 数据集处理

In [6]:
tokenier = AutoTokenizer.from_pretrained('D:/pretrained_model/models--hfl--chinese-macbert-base')

def process_function(examples):
    return tokenier(examples['completion'], max_length=384, padding=True, truncation=True)

In [7]:
tokenier_datasets = ds.map(process_function, batched=True, remove_columns=ds.column_names)
tokenier_datasets

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 10000
})

In [10]:
from torch.utils.data import DataLoader

dl = DataLoader(tokenier_datasets, batch_size=2, collate_fn=DataCollatorForLanguageModeling(tokenier, mlm=True, mlm_probability=0.15))

In [13]:
next(enumerate(dl))[1].keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [14]:
tokenier.mask_token, tokenier.pad_token, tokenier.mask_token_id, tokenier.pad_token_id

('[MASK]', '[PAD]', 103, 0)

## Step4 创建模型

In [17]:
model = AutoModelForMaskedLM.from_pretrained('D:/pretrained_model/models--hfl--chinese-macbert-base')

C:\Users\49207\AppData\Roaming\Python\Python311\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at D:/pretrained_model/models--hfl--chinese-macbert-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model 

## Step5 配置训练参数

In [18]:
args = TrainingArguments(
    output_dir='./masked_lm',
    per_device_train_batch_size=4,
    logging_steps=50,
    num_train_epochs=1
)

## Step6 创建训练器

In [21]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenier_datasets,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenier, mlm=True, mlm_probability=0.15)
)

C:\Users\49207\AppData\Roaming\Python\Python311\site-packages\accelerate\accelerator.py:447: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


## Step7 模型训练

In [22]:
trainer.train()

  0%|          | 0/2500 [00:00<?, ?it/s]

{'loss': 1.4621, 'grad_norm': 8.768919944763184, 'learning_rate': 4.9e-05, 'epoch': 0.02}
{'loss': 1.4938, 'grad_norm': 14.427478790283203, 'learning_rate': 4.8e-05, 'epoch': 0.04}
{'loss': 1.4778, 'grad_norm': 8.071401596069336, 'learning_rate': 4.7e-05, 'epoch': 0.06}
{'loss': 1.4871, 'grad_norm': 8.666216850280762, 'learning_rate': 4.600000000000001e-05, 'epoch': 0.08}
{'loss': 1.4847, 'grad_norm': 7.39116907119751, 'learning_rate': 4.5e-05, 'epoch': 0.1}
{'loss': 1.4882, 'grad_norm': 8.970457077026367, 'learning_rate': 4.4000000000000006e-05, 'epoch': 0.12}
{'loss': 1.4817, 'grad_norm': 7.982434272766113, 'learning_rate': 4.3e-05, 'epoch': 0.14}
{'loss': 1.4788, 'grad_norm': 10.128908157348633, 'learning_rate': 4.2e-05, 'epoch': 0.16}
{'loss': 1.4878, 'grad_norm': 8.277610778808594, 'learning_rate': 4.1e-05, 'epoch': 0.18}
{'loss': 1.4594, 'grad_norm': 8.80448055267334, 'learning_rate': 4e-05, 'epoch': 0.2}
{'loss': 1.5048, 'grad_norm': 8.660632133483887, 'learning_rate': 3.9000000

TrainOutput(global_step=2500, training_loss=1.397713217163086, metrics={'train_runtime': 548.1729, 'train_samples_per_second': 18.242, 'train_steps_per_second': 4.561, 'train_loss': 1.397713217163086, 'epoch': 1.0})

## Step8 模型推理

In [23]:
from transformers import pipeline

In [24]:
pipe = pipeline('fill-mask', model=model, tokenizer=tokenier, device=0)

In [ ]:
pipe("今天和工联院进行了[MASK]上的会议，彼此交换了信息，达成了一致的认识")

[{'score': 0.21726694703102112,
  'token': 3241,
  'token_str': '晚',
  'sequence': '今 天 和 工 联 院 进 行 了 晚 上 的 会 议 ， 彼 此 交 换 了 信 息 ， 达 成 了 一 致 的 认 识'},
 {'score': 0.0632537379860878,
  'token': 4408,
  'token_str': '班',
  'sequence': '今 天 和 工 联 院 进 行 了 班 上 的 会 议 ， 彼 此 交 换 了 信 息 ， 达 成 了 一 致 的 认 识'},
 {'score': 0.05488519370555878,
  'token': 809,
  'token_str': '以',
  'sequence': '今 天 和 工 联 院 进 行 了 以 上 的 会 议 ， 彼 此 交 换 了 信 息 ， 达 成 了 一 致 的 认 识'},
 {'score': 0.04968508705496788,
  'token': 3193,
  'token_str': '早',
  'sequence': '今 天 和 工 联 院 进 行 了 早 上 的 会 议 ， 彼 此 交 换 了 信 息 ， 达 成 了 一 致 的 认 识'},
 {'score': 0.03416558727622032,
  'token': 1963,
  'token_str': '如',
  'sequence': '今 天 和 工 联 院 进 行 了 如 上 的 会 议 ， 彼 此 交 换 了 信 息 ， 达 成 了 一 致 的 认 识'}]

In [26]:
pipe("下面是一则[MASK][MASK]新闻。小编报道，近日，游戏产业发展的非常好！")

[[{'score': 0.0623011514544487,
   'token': 7028,
   'token_str': '重',
   'sequence': '[CLS] 下 面 是 一 则 重 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'},
  {'score': 0.05762332305312157,
   'token': 2031,
   'token_str': '娱',
   'sequence': '[CLS] 下 面 是 一 则 娱 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'},
  {'score': 0.04219214990735054,
   'token': 4685,
   'token_str': '相',
   'sequence': '[CLS] 下 面 是 一 则 相 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'},
  {'score': 0.04130121320486069,
   'token': 3173,
   'token_str': '新',
   'sequence': '[CLS] 下 面 是 一 则 新 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'},
  {'score': 0.03979349136352539,
   'token': 3297,
   'token_str': '最',
   'sequence': '[CLS] 下 面 是 一 则 最 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'}],
 [{'score': 0.06800579279661179,
   'token': 7481,
   'token_str': '面',
   'sequence': '[CLS] 下 面 是 一 则 [MASK] 面 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'},
  {'